In [38]:
import pandas as pd
import math

In [39]:
def base_entropy(dataset):
    p = 0
    n = 0
    target = dataset.iloc[:, -1] # get the target column
    targets = list(set(target)) # set is used to get the categorical values, and use list to make an array
    for i in target:
        if i == targets[0]:
            p = p + 1
        else:
            n = n + 1
    if p == 0 or n == 0: # if one of it has already been divided
        return 0
    elif p == n: # if all the categories have equal chances - entropy = 1
        return 1
    else:
        entropy = 0 - (
            ((p / (p + n)) * (math.log2(p / (p + n))) + (n / (p + n)) * (math.log2(n / (p + n)))))
        return entropy

In [40]:
def entropy(dataset, feature, attribute):
    p = 0
    n = 0
    target = dataset.iloc[:, -1]
    targets = list(set(target))
    for i, j in zip(feature, target):
        if i == attribute and j == targets[0]:
            p = p + 1
        elif i == attribute and j == targets[1]:
            n = n + 1
    if p == 0 or n == 0:
        return 0
    elif p == n:
        return 1
    else:
        entropy = 0 - (
            ((p / (p + n)) * (math.log2(p / (p + n))) + (n / (p + n)) * (math.log2(n / (p + n)))))
        return entropy

In [41]:
def counter(target, attribute, i):
    p = 0
    n = 0
    targets = list(set(target))
    for j, k in zip(target, attribute):
        if j == targets[0] and k == i:
            p = p + 1
        elif j == targets[1] and k == i:
            n = n + 1
    return p, n

In [42]:
def Information_Gain(dataset, feature):
    Distinct = list(set(feature))
    Info_Gain = 0
    for i in Distinct:
        Info_Gain = Info_Gain + feature.count(i) / len(feature) * entropy(dataset, feature, i)
    Info_Gain = base_entropy(dataset) - Info_Gain
    return Info_Gain

In [43]:
def generate_childs(dataset, attribute_index):
    distinct = list(dataset.iloc[:, attribute_index])
    childs = dict()
    for i in distinct:
        childs[i] = counter(dataset.iloc[:, -1], dataset.iloc[:, attribute_index], i)
    return childs

In [44]:
def modify_data_set(dataset,index, feature, impurity):
    size = len(dataset)
    subdata = dataset[dataset[feature] == impurity]
    del (subdata[subdata.columns[index]])
    return subdata

In [45]:
def greatest_information_gain(dataset):
    max = -1
    attribute_index = 0
    size = len(dataset.columns) - 1
    for i in range(0, size):
        feature = list(dataset.iloc[:, i])
        i_g = Information_Gain(dataset, feature)
        if max < i_g:
            max = i_g
            attribute_index = i
    return attribute_index

In [46]:
def construct_tree(dataset, tree):
    target = dataset.iloc[:, -1]
    impure_childs = []
    attribute_index = greatest_information_gain(dataset)
    childs = generate_childs(dataset, attribute_index)
    tree[dataset.columns[attribute_index]] = childs
    targets = list(set(dataset.iloc[:, -1]))
    for k, v in childs.items():
        if v[0] == 0:
            tree[k] = targets[1]
        elif v[1] == 0:
            tree[k] = targets[0]
        elif v[0] != 0 or v[1] != 0:
            impure_childs.append(k)
    for i in impure_childs:
        sub = modify_data_set(dataset,attribute_index, dataset.columns[attribute_index], i)
        tree = construct_tree(sub, tree)
    return tree

In [47]:
def main():
    df = pd.read_csv("data1.csv")
    tree = dict()
    result = construct_tree(df, tree)
    for key, value in result.items():
        print(key, " => ", value)

In [48]:
if __name__ == "__main__":
    main()

Outlook  =>  {'Sunny': (0, 5), 'Rainy': (3, 2)}
Sunny  =>  Yes
Company  =>  {'No': (2, 0), 'Medium': (1, 1), 'Big': (0, 1)}
No  =>  No
Big  =>  Yes
Sailboat  =>  {'Small': (0, 1), 'Big_S': (1, 0)}
Small  =>  Yes
Big_S  =>  No


In [77]:
df

,Outlook,Company,Sailboat,Sail
0,Sunny,Big,Small,Yes
1,Sunny,Medium,Small,Yes
2,Sunny,Medium,Big_S,Yes
3,Sunny,No,Small,Yes
4,Sunny,Big,Big_S,Yes
5,Rainy,No,Small,No
6,Rainy,Medium,Small,Yes
7,Rainy,Big,Big_S,Yes
8,Rainy,No,Big_S,No
9,Rainy,Medium,Big_S,No
